In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

spark = SparkSession.builder \
    .appName("Final_Project") \
    .getOrCreate()

sc = spark.sparkContext
spark

24/09/03 13:47:58 WARN Utils: Your hostname, DESKTOP-26AECPL resolves to a loopback address: 127.0.1.1; using 192.168.220.1 instead (on interface eth1)
24/09/03 13:47:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/03 13:47:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df1 = spark.read.parquet("/home/codebind/spark/Final Project_Spark/VertFinalExam")
df1.show()

24/09/03 13:48:13 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+---+-------------+-----+-------+
| id|         City|State|Country|
+---+-------------+-----+-------+
|ABE|    Allentown|   PA|    USA|
|ABI|      Abilene|   TX|    USA|
|ABQ|  Albuquerque|   NM|    USA|
|ABR|     Aberdeen|   SD|    USA|
|ABY|       Albany|   GA|    USA|
|ACK|    Nantucket|   MA|    USA|
|ACT|         Waco|   TX|    USA|
|ACV|       Eureka|   CA|    USA|
|ACY|Atlantic City|   NJ|    USA|
|ADQ|       Kodiak|   AK|    USA|
|AEX|   Alexandria|   LA|    USA|
|AGS|      Augusta|   GA|    USA|
|AHN|       Athens|   GA|    USA|
|AIA|     Alliance|   NE|    USA|
|AKN|  King Salmon|   AK|    USA|
|ALB|       Albany|   NY|    USA|
|ALO|     Waterloo|   IA|    USA|
|ALS|      Alamosa|   CO|    USA|
|ALW|  Walla Walla|   WA|    USA|
|AMA|     Amarillo|   TX|    USA|
+---+-------------+-----+-------+
only showing top 20 rows



In [3]:
df1.printSchema()

root
 |-- id: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)



In [4]:
df2 = spark.read.parquet("/home/codebind/spark/Final Project_Spark/EdgesFinalExam")
df2.show()

+-------+-----+--------+---+---+
| tripid|delay|distance|src|dst|
+-------+-----+--------+---+---+
|1010630|  -10|     928|RSW|EWR|
|1021029|   87|     974|RSW|ORD|
|1021346|    0|     928|RSW|EWR|
|1021044|   18|     928|RSW|EWR|
|1021730|   29|     748|RSW|IAH|
|1020535|  605|     974|RSW|ORD|
|1021820|   71|     974|RSW|ORD|
|1021743|    0|     928|RSW|EWR|
|1022017|    0|     928|RSW|EWR|
|1020600|   -2|     748|RSW|IAH|
|1021214|   29|     891|RSW|CLE|
|1020630|   -5|     928|RSW|EWR|
|1031029|   13|     974|RSW|ORD|
|1031346|  279|     928|RSW|EWR|
|1031740|   29|     748|RSW|IAH|
|1030535|    0|     974|RSW|ORD|
|1031808|   -3|     974|RSW|ORD|
|1031516|   -2|    1396|RSW|DEN|
|1032017|   14|     928|RSW|EWR|
|1031214|   17|     891|RSW|CLE|
+-------+-----+--------+---+---+
only showing top 20 rows



In [5]:
df2.printSchema()

root
 |-- tripid: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)



In [6]:
from pyspark.sql.types import *

vertices_schema = StructType([
    StructField("id", StringType(), True),
    StructField("City", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Country", StringType(), True)
])

edges_schema = StructType([
    StructField("tripid", IntegerType(), True),
    StructField("delay", IntegerType(), True),
    StructField("distance", IntegerType(), True),
    StructField("src", StringType(), True),
    StructField("dst", StringType(), True)
    
])


In [7]:
df_vertices = spark.readStream \
    .format("parquet") \
    .option("header", "False") \
    .schema(vertices_schema) \
    .load("/home/codebind/spark/Final Project_Spark/VertFinalExam")  

In [8]:
df_edges = spark.readStream \
    .format("parquet") \
    .option("header", "False") \
    .schema(edges_schema) \
    .load("/home/codebind/spark/Final Project_Spark/EdgesFinalExam")  

In [9]:
from pyspark.sql.functions import when

df_edges_processed = df_edges.withColumn(
    "delay_category",
    when(df_edges["delay"] < 0, "Early")
    .when(df_edges["delay"] > 0, "Late")
    .otherwise("OnTime")
)

In [10]:
df_vertices_processed = df_vertices.filter(df_vertices["state"] != "")

In [11]:
vertices_query = df_vertices_processed.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "/home/codebind/spark/Final Project_Spark/MyFirstFolder") \
    .option("checkpointLocation", "/home/codebind/spark/Final Project_Spark/MyFirstFolder/vertices_checkpoint") \
    .trigger(processingTime="10 seconds") \
    .start()

24/09/03 13:48:16 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [12]:
edges_query = df_edges_processed.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "/home/codebind/spark/Final Project_Spark/MySecondFolder") \
    .option("checkpointLocation", "/home/codebind/spark/Final Project_Spark/MySecondFolder/edges_checkpoint") \
    .trigger(processingTime="10 seconds") \
    .start()

24/09/03 13:48:16 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [13]:
import time
time.sleep(60)

In [14]:
vertices_query.stop()

In [15]:
edges_query.stop()

In [18]:
static_vertices_df = spark.read.format("parquet") \
    .load("/home/codebind/spark/Final Project_Spark/MyFirstFolder")

static_vertices_df.show()

+---+-------------+-----+-------+
| id|         City|State|Country|
+---+-------------+-----+-------+
|ABE|    Allentown|   PA|    USA|
|ABI|      Abilene|   TX|    USA|
|ABQ|  Albuquerque|   NM|    USA|
|ABR|     Aberdeen|   SD|    USA|
|ABY|       Albany|   GA|    USA|
|ACK|    Nantucket|   MA|    USA|
|ACT|         Waco|   TX|    USA|
|ACV|       Eureka|   CA|    USA|
|ACY|Atlantic City|   NJ|    USA|
|ADQ|       Kodiak|   AK|    USA|
|AEX|   Alexandria|   LA|    USA|
|AGS|      Augusta|   GA|    USA|
|AHN|       Athens|   GA|    USA|
|AIA|     Alliance|   NE|    USA|
|AKN|  King Salmon|   AK|    USA|
|ALB|       Albany|   NY|    USA|
|ALO|     Waterloo|   IA|    USA|
|ALS|      Alamosa|   CO|    USA|
|ALW|  Walla Walla|   WA|    USA|
|AMA|     Amarillo|   TX|    USA|
+---+-------------+-----+-------+
only showing top 20 rows



In [19]:
static_edges_df = spark.read.format("parquet") \
    .load("/home/codebind/spark/Final Project_Spark/MySecondFolder")

static_edges_df.show()

+-------+-----+--------+---+---+--------------+
| tripid|delay|distance|src|dst|delay_category|
+-------+-----+--------+---+---+--------------+
|1010630|  -10|     928|RSW|EWR|         Early|
|1021029|   87|     974|RSW|ORD|          Late|
|1021346|    0|     928|RSW|EWR|        OnTime|
|1021044|   18|     928|RSW|EWR|          Late|
|1021730|   29|     748|RSW|IAH|          Late|
|1020535|  605|     974|RSW|ORD|          Late|
|1021820|   71|     974|RSW|ORD|          Late|
|1021743|    0|     928|RSW|EWR|        OnTime|
|1022017|    0|     928|RSW|EWR|        OnTime|
|1020600|   -2|     748|RSW|IAH|         Early|
|1021214|   29|     891|RSW|CLE|          Late|
|1020630|   -5|     928|RSW|EWR|         Early|
|1031029|   13|     974|RSW|ORD|          Late|
|1031346|  279|     928|RSW|EWR|          Late|
|1031740|   29|     748|RSW|IAH|          Late|
|1030535|    0|     974|RSW|ORD|        OnTime|
|1031808|   -3|     974|RSW|ORD|         Early|
|1031516|   -2|    1396|RSW|DEN|        